# Porcupine Neural Networks

The code here is for testing PNN approximation on MNIST (only 0's and 1's).

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
%load_ext autoreload
%autoreload 2

from __future__ import print_function

import os
import time
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import arch
import dl_utils
import utils

np.set_printoptions(precision=2,suppress=True)
gpu_id = 0

%matplotlib inline

maindir = './temp_PNN/'

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

# subset to just 1's and 0's

mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

keep_inds_train = (mnist.train.labels == 0) | (mnist.train.labels == 1)
Xtr = mnist.train.images[keep_inds_train]
Ytr = mnist.train.labels[keep_inds_train].astype(float).reshape(-1, 1)
Ytr[Ytr == 0] = -1

keep_inds_test = (mnist.test.labels == 0) | (mnist.test.labels == 1)
Xtt = mnist.test.images[keep_inds_test]
Ytt = mnist.test.labels[keep_inds_test].astype(float).reshape(-1, 1)
Ytt[Ytt == 0] = -1

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# Same as graph_builder_wrapper except for sparser structures
# CURRENTLY ONLY WORKS WITH mlp_noreluout_lastlayernottrainable
def graph_builder_wrapper_PNN(input_dict, build_func=arch.mlp, lr_initial=0.01, max_save=100):
    graph = build_func(input_dict)
    
    # Loss
    y = tf.placeholder(tf.float32, shape=[None,1])
    total_loss = dl_utils.loss(y, graph['yhat'])
    
    # W gap
    w = {k:tf.placeholder(tf.float32, shape=[None,None]) for k in graph if 'weights' in k}
    total_w_gap = dl_utils.w_gap(w, {k:graph[k] for k in graph if 'weights' in k})
    
    # Find unit vectors
    u = graph['weights1'] / tf.sqrt(tf.reduce_sum(tf.square(graph['weights1']), 0, keep_dims=True))
    
    # Optimizer
    learning_rate = tf.Variable(lr_initial, name='learning_rate')
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads = optimizer.compute_gradients(total_loss)
    modified_grads = []
    for gv in grads:
        if gv[0] is None: modified_grads.append(gv)
        else: 
            # Project gradients along unit vectors
            alpha = tf.diag_part(tf.matmul(tf.transpose(u),gv[0]))
            update = u*alpha
            modified_grads.append((update, gv[1]))
            
    opt_step = optimizer.apply_gradients(modified_grads)
    
    graph['y'] = y
    graph['w'] = w
    graph['u'] = u
    graph['grads'] = [i for i in grads if i[0] is not None]
    graph['modified_grads'] = [i for i in modified_grads if i[0] is not None]
    graph['opt_step'] = opt_step
    graph['total_loss'] = total_loss
    graph['total_w_gap'] = total_w_gap
    graph['learning_rate'] = learning_rate
    graph['saver'] = tf.train.Saver(max_to_keep=100)
    return graph

# Used to check to make sure final weights are aligned with initial weights
def check_to_see_if_col_are_same_line(X1,X2,verbose=True):
    u1 = X1 / np.linalg.norm(X1,axis=0)
    u2 = X2 / np.linalg.norm(X2,axis=0)
    dot_products = np.abs(np.diag(np.dot(u1.T,u2)))
    if verbose:
        for i in dot_products: print(i)
        if np.all(np.isclose(dot_products,1)):
            print('columns along same line')
        else:
            print('columns not along same line')            
    return dot_products

def get_acc(X, Y, w, input_dict, build_func):
    Yhat = dl_utils.generate_output(X, w, input_dict, build_func=build_func)
    return np.sum(np.sign(Yhat) == Y)/float(len(Y))

In [5]:
num_scale_redraws = 10
N = 10000      # number of samples

# Network parameters
num_epochs = 100
batch_size = 100
build_func = arch.mlp_noreluout_lastlayernottrainable
d = 784    # number of inputs

In [6]:
# Performance with a dense network
input_dict = dict(p=d, h=512)
tf.reset_default_graph()
g = dl_utils.graph_builder_wrapper(input_dict, build_func=build_func)
out = dl_utils.train_no_wtrue(Xtr, Ytr, g, num_epochs, batch_size)
pickle.dump(out, file('temp_MNIST/mlp_noreluout_lastlayernottrainable.pickle', 'wb'))

tr_acc = get_acc(Xtr, Ytr, out[1], input_dict, build_func)
test_acc = get_acc(Xtt, Ytt, out[1], input_dict, build_func)

print('Train acc: %.5f'%(tr_acc))
print('Test acc: %.5f'%(test_acc))

Epoch 100/100 (380.127 s), batch 117.0/116 (3.918 s): loss 0.006
Train acc: 0.99966
Test acc: 0.99905


In [ ]:
num_epochs = 500

# Performance with a PNN network
for k in [512, 1024, 2048]:
    
    input_dict = dict(p=d, h=k)
    
    # initialize unit vectors corresponding to lines
    w_init_u = np.random.normal(0, 1, [d, k/2])
    w_init_u /= np.linalg.norm(w_init_u, axis=0)
    w_init_u = np.repeat(w_init_u, 2, axis=1)
    alpha = np.random.uniform(0, 1, k)
    for ii in range(0, len(alpha), 2): alpha[ii] *= -1
    w_init = alpha*w_init_u
    w_init = {'weights1':w_init.astype(np.float32),
              'weights2':np.ones([k, 1]).astype(np.float32)}
    
    # start training
    tf.reset_default_graph()
    g = graph_builder_wrapper_PNN(input_dict, build_func=build_func)
    out = dl_utils.train_no_wtrue(Xtr, Ytr, g, num_epochs, batch_size, w_initial=w_init)
    pickle.dump(out, file('temp_MNIST/mlp_noreluout_lastlayernottrainable_PNN_hidden%s.pickle'%(k), 'wb'))

    dot_prods = check_to_see_if_col_are_same_line(out[1]['weights1'], w_init['weights1'], verbose=False)
    
    tr_acc = get_acc(Xtr, Ytr, out[1], input_dict, build_func)
    test_acc = get_acc(Xtt, Ytt, out[1], input_dict, build_func)
    
    print('Num final lines close to original: %s'%(np.sum(np.isclose(dot_prods, np.ones(k), atol=1e-3))))
    print('Train acc: %.5f'%(tr_acc))
    print('Test acc: %.5f'%(test_acc))

Epoch 358/500 (2021.963 s), batch 90.0/116 (4.574 s): loss 0.5523

In [ ]:
# Plot all losses
loss1 = pickle.load(file('temp_MNIST/mlp_noreluout_lastlayernottrainable.pickle', 'rb'))[0]
loss2 = pickle.load(file('temp_MNIST/mlp_noreluout_lastlayernottrainable_PNN_hidden512.pickle', 'rb'))[0]
loss3 = pickle.load(file('temp_MNIST/mlp_noreluout_lastlayernottrainable_PNN_hidden1024.pickle', 'rb'))[0]
loss4 = pickle.load(file('temp_MNIST/mlp_noreluout_lastlayernottrainable_PNN_hidden2048.pickle', 'rb'))[0]

plt.figure(figsize=(10, 7))
plt.plot(loss1, label='dense, 512 hidden')
plt.plot(loss2, label='PNN, 512 hidden')
plt.plot(loss3, label='PNN, 1024 hidden')
plt.plot(loss4, label='PNN, 2048 hidden')
plt.xlabel('epoch')
plt.ylabel('loss (MSE)')
plt.grid()
plt.yscale('log')
plt.show()